# Speed of computation

Python, being an interpreted language, tends to be slower than compiled languages like C or Fortran.  Some other languages like Java and Julia tend to use Just-in-Time compilation which can give speedups, but Python also has the problem of being dynamically typed, which eliminates the possibility of many optimizations.

The `timeit` library provides functions to estimate the time taken to run a piece of code.  It can automatically run the code multiple times to get better average results, and can be used to identify bottlenecks in your program.  However, it should be used with care as it is not a detailed function-call-level profiler.

It can either be `import`ed as a module where you can then explicitly called `timeit.timeit(func)` to estimate time for a function, or you can use the *magic syntax* in Python notebooks as shown below.  

import numpy as np
x = np.random.rand(10000,1)

In [28]:
def sumarr(x):
    sum = 0
    for i in range(len(x)): 
    # for i in x:
        sum += x[i]
    return sum
x = [1,5,7]
print(sumarr(x))
%timeit sumarr(x)

13
251 ns ± 11.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [29]:
import numpy as np
def npsumarr(x):
    return np.sum(x)
print(npsumarr(x))
%timeit npsumarr(x)

13
4.22 µs ± 119 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# Solving equations by Gaussian elimination

Once you have constructed two matrices A and B to represent the system of linear equations 
$$ Ax = b $$
you can then proceed to solve the equations using the process known as Gaussian elimination.

It is assumed you already know how the process works, but to refresh your memory, you could use the reference material at [LibreTexts](https://math.libretexts.org/Bookshelves/Algebra/Book%3A_Algebra_and_Trigonometry_(OpenStax)/11%3A_Systems_of_Equations_and_Inequalities/11.06%3A_Solving_Systems_with_Gaussian_Elimination).

Basically it involves making the A matrix *triangular* and ultimately into the shape of an identity matrix.

In [30]:
# Input matrices - the set of equations - 2 variables x1 and x2
A = [ [2,3], [1,-1] ]
B = [6,1/2]
print(A)
print(B)

[[2, 3], [1, -1]]
[6, 0.5]


In [31]:
# Normalize row 1
norm = A[0][0]
for i in range(len(A[0])): A[0][i] /= norm
B[0] = B[0]/norm
print(A)
print(B)

[[1.0, 1.5], [1, -1]]
[3.0, 0.5]


In [32]:
# Eliminate row 2 - A[1] - need to check and ensure div-by-zero etc doesnt happen
norm = A[1][0] / A[0][0]
for i in range(len(A[1])): A[1][i] = A[1][i] - A[0][i] * norm
B[1] = B[1] - B[0] * norm
print(A)
print(B)

[[1.0, 1.5], [0.0, -2.5]]
[3.0, -2.5]


In [33]:
# Normalize row 2 - B[1] will now contain the solution for x2
norm = A[1][1]
for i in range(len(A[1])): A[1][i] = A[1][i] / norm
B[1] = B[1] / norm
print(A)
print(B)

[[1.0, 1.5], [-0.0, 1.0]]
[3.0, 1.0]


In [34]:
# Sub back and solve for B[0] <-> x1
# This can be seen as eliminating A[0][1]
norm = A[0][1] / A[0][0]
# note that len(A) will give number of rows
for i in range(len(A)): 
    A[i][1] = A[i][1] - A[i][0] * norm
    B[i] = B[i] - A[i][0] * norm
print(A)
print(B)

[[1.0, 0.0], [-0.0, 1.0]]
[1.5, 1.0]


## Problems with Gaussian elimination

There are several obvious problems with the method outlined here.  These include:

- Performance - Python lists are not the most efficient way to store matrices
- Zeros: the simple example does not consider a scenario where one of the values on the diagonal may be 0.  Then some shuffling of rows is required.
- Numerical stability: there are several *normalization* steps involved, where it is quite possible for the values to blow up out of control if not managed properly.  Usually some kind of pivoting techniques are used to get around these issues.

In [35]:
import numpy as np
A1 = np.array( [ [2,3], [1,-1] ] )
B1 = np.array( [6, 1/2] )
np.linalg.solve(A1, B1)

array([1.5, 1. ])

# Library functions

*Numeric Python* or `numpy` is a library that allows Python code to directly call highly efficient implementations of several linear algebra routines (that have been written and optimized using C or Fortran and generally offer very high performance).

Although you can use the same `list` based approach to create matrices, it is better to declare them as the `array` data type:

- the numeric `type` (float, int etc.) can be specified for arrays
- memory allocation is done more efficiently by assuming they are not meant to be arbitrarily extended or changed

# SPICE basics

Our goal is to implement a SPICE simulator.  In order to do this, we first need to read in the circuit description from a text file.  To start with, we will only consider the basic elements of SPICE: Voltage sources, Current sources, and Resistors.  A typical SPICE netlist would look like this:

```spice
.circuit
R1 GND 1 1  
R2 1 2 1    
V1 GND 2 dc 2
.end
```

This is basically a *netlist* with 3 *nodes* - one of which is Ground (GND) which is assumed to be have a voltage of 0V.  We can write down Kirchhoff's current law (KCL) equations at each node, to account for current balance.  In addition, we will have some equations that specify the voltages between nodes having a direct voltage source, since there is no resistance there to provide an equation.

For the above example, the equations will be

$$
\begin{aligned}
\frac{V1-0}{R1} & + & \frac{V1-V2}{R2} & = & 0 \\
\frac{V2-V1}{R2} & + & I1 & = & 0 \\
V2 & - & 0 & = & 2
\end{aligned}
$$

which can be written in Matrix form as:

$$
\begin{bmatrix}
\frac{1}{R1}+\frac{1}{R2} & \frac{-1}{R2} & 0 \\
\frac{-1}{R2}   & \frac{1}{R2}  & 1 \\
0  & 1  & 0
\end{bmatrix}
\begin{bmatrix}
V1 \\
V2 \\
I1
\end{bmatrix}
=
\begin{bmatrix}
0 \\
0 \\
2
\end{bmatrix}
$$

At this point, you have reduced the solving of the SPICE equations to a known problem (linear equation solving) that you already know how to do.

## AC sources

The assumption above is that the system consists only of Voltage or Current sources and resistors.  What about capacitors, inductors, and AC sources?  These can be handled in exactly the same way as long as the circuit is operating at a single frequency.  We then replace the elements with their corresponding *impedance* values, which are frequency dependent complex numbers, but since there is only a single frequency they will still be constants.

## Problem scenarios

- Voltage source loops
- Current sources at a node
- Circuit defined with both DC and AC sources
- Syntax errors

# String and File manipulation

Given a SPICE netlist like the one above, you need to *read* it and construct an internal matrix as described above.  For string manipulation, there are a few helpful utility functions that we can see here.

In [36]:
circ = """.circuit
R1 GND 1 1  
R2 1 2 1    
V1 GND 2 dc 2
.end
""".splitlines()
for l in circ:
    if l[0] == 'R':
        print("Found a resistor")
    elif l[0] == 'V':
        print("Found a voltage source with value: ", float(l.split()[4]))

Found a resistor
Found a resistor
Found a voltage source with value:  2.0


## Files

You can read from a file using the `readlines()` method of file objects.  One thing to keep in mind is how you open and close file objects.  In particular, it is strongly recommended to use the pattern `with open("filename") as f:` to ensure that the file is closed once you are done with reading it.  

# Assignment

The following are the problems you need to solve for this assignment.  You need to submit your code (either as standalone Python script or a Python notebook), a PDF document explaining your solution (either generated from the notebook or a separate LaTeX document), and any supporting files you may have (such as circuit netlists you used for testing your code).

- Write a function to find the factorial of N (N being an input) and find the time taken to compute it.  This will obviously depend on where you run the code and which approach you use to implement the factorial.  Explain your observations briefly.
- Write a linear equation solver that will take in matrices $A$ and $b$ as inputs, and return the vector $x$ that solves the equation $Ax=b$.  Your function should catch errors in the inputs and return suitable error messages for different possible problems.
  - Time your solver to solve a random $10\times 10$ system of equations.  Compare the time taken against the `numpy.linalg.solve` function for the same inputs.
- Given a circuit netlist in the form described above, read it in from a file, construct the appropriate matrices, and use the solver you have written above to obtain the voltages and currents in the circuit.  If you find AC circuits hard to handle, first do this for pure DC circuits, but you should be able to handle both voltage and current sources.

## Bonus assignments

- (Small bonus): after reading in the netlist, allow some or all sources and impedances to be controlled interactively - either using widgets or other mechanisms.  On each change you should recompute the currents and voltages and display them.
- (Large bonus): make a solver that can do real-time transient simulations of a SPICE netlist and update the currents and voltages dynamically.  They should also be plotted as a function of time and react to changes.  This is something along the lines of https://www.falstad.com/circuit/.  Ideally you should be able to do a real-time demo of some experiments you might conduct as part of a basic electronics lab, and simulate the behaviour of an oscilloscope and signal generator.

In [37]:
# Question 1:
def fact(x):
    if x == 0:
        return 1
    else:
        return x * fact(x-1)
try:
    x = int(input("Enter a number to find its factorial"))
    if x<0:
        print("Enter an positive integer as input")
    else:
        print(fact(x))
        %timeit fact(x)
        import numpy as np
        def npfactorial(x):
            return np.math.factorial(x)
        %timeit npfactorial(x)
except ValueError:
    print("Enter an positive integer as input")

Enter a number to find its factorial 7


5040
614 ns ± 34.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
132 ns ± 12.4 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Explanation :
<br> Here, I have used recursion method to find the factorialof thegiven number. It multiplies the given number with its preceeding number until 1. This gives us factorial. I also used numpy method to find the factorial and I have tried to compare the time taken in both cases. Afther observing some test cases I can observe that numpy method takes less time in completing the process and the time difference increases when we try to give higher numbers. So, It is effecient to use numpy in these types of programs.

In [38]:
c=int(input("Enter no of colomns"))
r=int(input("Enter no of rows"))
def swap(t,AB,n):
    for i in range(n):
        if AB[i][t]!=0:
            AB[t], AB[i] = AB[i], AB[t]
            break

def guassian(A,B):
    n=len(A)
    AB = []
    for i in range(n):
        AB.append(A[i] + [B[i]])
    for t in range(n):
        if AB[t][t] == 0:swap(t,AB,n)
        norm = AB[t][t]
        for i in range(t, n+1):
            AB[t][i]=AB[t][i]/norm
        for i in range(n):
            if i == t:continue
            norm = AB[i][t]
            for j in range(t, n+1):
                AB[i][j]=AB[i][j]- norm * AB[t][j]
    for i in range(n):
        if AB[i][i]==0 and AB[i][n]==0:
            return "The given system of linear equations has infinite solutions"
        elif AB[i][i]==0 and AB[i][n]!=0:
            return "The given system of linear equations has no solution"
    ans = [AB[i][n] for i in range(n)]
    return ans

A=[]
while c!= r:
    c = int(input("Enter no of colomns: "))
    r = int(input("Enter no of rows: "))
try:
    for i in range(c):
        print(f"Enter the elements of A {i+1},where each coefficient is space separated")
        row=[complex(x) for x in input().split()]
        A.append(row)
    print("Enter B's elements , space separated")
    B = [complex(x) for x in input().split()]
    x = np.linalg.solve(A,B)
    print(x)
    print(guassian(A,B))
except ValueError:
        print("please give a floating number")

Enter no of colomns 2
Enter no of rows 2


Enter the elements of A 1,where each coefficient is space separated


 1 2


Enter the elements of A 2,where each coefficient is space separated


 5 6


Enter B's elements , space separated


 3 1


[-4. +0.j  3.5+0.j]
[(-4+0j), (3.5-0j)]


Explanation :
<br> I have written a programme that is capable of doing guass-elimination method of solving the linear equations.<br>
I have made matrix A and B as a single matrix AB(agumented matrix)<br>
If a diagonal element is Zero the I have swapped the that row with other row which does not contain zero in that element<br>
First it tries to make A as a Identity matrix and checks for inconsistency<br>
Later the The colomn which was made with B is returned as answer.<br>
I have later checked with the numpy method to verify .


In [39]:
import random
A=[]
for i in range(10):
    A.append([random.randint(1, 20) for i in range(10)])
B=[random.randint(1, 200) for i in range(10)]

x=np.linalg.solve(A,B)
print(x)
%timeit np.linalg.solve(A,B)
print(guassian(A,B))
%timeit guassian(A,B)

[  8.34927624  20.9919545   -6.63263586   3.40864076 -14.15383893
   1.29537548  -1.79608589   6.58088701   4.80956235  -4.94703097]
27.2 µs ± 751 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
[8.349276236333864, 20.991954504870918, -6.632635858066267, 3.4086407597249107, -14.15383893418726, 1.2953754813835872, -1.7960858850872912, 6.5808870080483635, 4.809562346527379, -4.947030971265774]
79 µs ± 591 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


I have taken a random 10 X 10 Matrix<br>
And after that I have solved for the equations in two different ways and tested the time taken<br>
As we can see the numpy method is effecient in Solving linear equations.